In [820]:
import pandas as pd
import numpy as np
import re
from razdel import tokenize as razdel_tokenize
from string import punctuation
from pymystem3 import Mystem
from stop_words import get_stop_words
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from ftfy import fix_text
from IPython.display import Image
from IPython.core.display import HTML 

In [471]:
russian_stopwords = get_stop_words('ru')
punctuation += '«―_–»'

# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [539]:
data = pd.read_csv('labeled.csv')

In [540]:
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


Дефолтный векторизатор

In [847]:
x_train, x_test, y_train, y_test = train_test_split(data['comment'], data['toxic'], test_size=0.1, shuffle=True, stratify=data['toxic'], random_state=42)

In [848]:
vectorizer = CountVectorizer(stop_words=russian_stopwords)
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [849]:
lr = LogisticRegression(random_state=42)
lr.fit(x_train, y_train)

LogisticRegression(random_state=42)

In [850]:
y_pred = lr.predict(x_test)

In [851]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.95      0.88       959
         1.0       0.85      0.59      0.69       483

    accuracy                           0.83      1442
   macro avg       0.83      0.77      0.79      1442
weighted avg       0.83      0.83      0.82      1442



In [852]:
lr = LogisticRegression(random_state=42)
cvs = cross_val_score(lr, x_train, y_train, scoring='f1')

In [853]:
cvs.mean()

0.7000806993927755

Собственный векторизатор

In [839]:
x_train, x_test, y_train, y_test = train_test_split(data['comment'], data['toxic'], test_size=0.1, shuffle=True, stratify=data['toxic'], random_state=42)

In [840]:
def my_tokenizer(text):
    return [token.text.lower() for token in list(razdel_tokenize(text)) if token.text.lower() not in punctuation]

In [841]:
vectorizer = CountVectorizer(stop_words=russian_stopwords, tokenizer=my_tokenizer)
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [842]:
lr = LogisticRegression(random_state=42)
lr.fit(x_train, y_train)

LogisticRegression(random_state=42)

In [843]:
y_pred = lr.predict(x_test)

In [844]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.94      0.88       959
         1.0       0.84      0.60      0.70       483

    accuracy                           0.83      1442
   macro avg       0.83      0.77      0.79      1442
weighted avg       0.83      0.83      0.82      1442



In [845]:
lr = LogisticRegression()
cvs = cross_val_score(lr, x_train, y_train, scoring='f1')

In [846]:
cvs.mean()

0.7001020422819904

Разницы почти нет, разве что f1-score для токсичных комментариев выше на 0.1.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [589]:
rows = {'я и ты': {'я': 1, 'ты': 1, 'и': 1, 'только': 0, 'не': 0, 'он': 0}, 
        'ты и я': {'я': 1, 'ты': 1, 'и': 1, 'только': 0, 'не': 0, 'он': 0}, 
        'я, я и только я': {'я': 3, 'ты': 0, 'и': 1, 'только': 1, 'не': 0, 'он': 0}, 
        'только не я': {'я': 1, 'ты': 0, 'и': 0, 'только': 1, 'не': 1, 'он': 0}, 
        'он': {'я': 0, 'ты': 0, 'и': 0, 'только': 0, 'не': 0, 'он': 1}}

In [590]:
tokenized_rows = [[token for token in my_tokenizer(sent) if token not in punctuation] for sent in rows.keys()]

In [591]:
token_df = {}

for sent in tokenized_rows:
    for token in set(sent):
        if token in token_df:
            token_df[token] += 1
        else:
            token_df[token] = 1

In [592]:
for string, keywords in rows.items():
    for word, freq in keywords.items():
        idf = len(keywords) / token_df[word]
        tf = freq / sum(keywords.values())
        tfidf = tf * idf
        keywords[word] = tfidf

In [593]:
pd.DataFrame.from_dict(rows, orient='index')

,я,ты,и,только,не,он
я и ты,0.5,1.2,0.740741,0.000000,0.000000,0.0
ты и я,0.5,1.2,0.740741,0.000000,0.000000,0.0
"я, я и только я",0.9,0.0,0.689655,1.158455,0.000000,0.0
только не я,0.5,0.0,0.000000,1.200000,2.222222,0.0
он,0.0,0.0,0.000000,0.000000,0.000000,6.0


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [854]:
data = pd.read_csv('labeled.csv')

In [855]:
data['comment'] = data['comment'].apply(lambda x: re.sub('\s+', ' ', x).strip())

In [856]:
x_train, x_test, y_train, y_test = train_test_split(data['comment'], data['toxic'], test_size=0.1, shuffle=True, stratify=data['toxic'], random_state=42)

tfidf_vectorizer = TfidfVectorizer(stop_words=russian_stopwords, 
                                   ngram_range=(1,2),
                                   max_df=0.999,
                                   min_df=0.001,
                                   tokenizer=my_tokenizer,
                                   max_features=2000,
                                   sublinear_tf=True)

x_train_vect = tfidf_vectorizer.fit_transform(x_train)
x_test_vect = tfidf_vectorizer.transform(x_test)

In [857]:
lr = LogisticRegression(C=1e-1, 
                        random_state=42,
                        class_weight='balanced',
                        solver='newton-cg')

lr.fit(x_train_vect, y_train)
y_preds = lr.predict(x_test_vect)

In [858]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       959
         1.0       0.68      0.78      0.73       483

    accuracy                           0.81      1442
   macro avg       0.78      0.80      0.79      1442
weighted avg       0.82      0.81      0.81      1442



In [859]:
x_test = x_test.reset_index()
x_test['pred_proba'] = lr.predict_proba(x_test_vect)[:, 1]

In [860]:
x_test.drop(['index'], axis=1, inplace=True)

In [861]:
toxic_texts_lr = x_test.sort_values(by='pred_proba', ascending=False).head(10).values

for text, prob in toxic_texts_lr:
    print('Текст: ', text)
    print('Вероятность токсичности', round(prob, 2))
    print()

Текст:  БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. НЕПРОБИВАЕМАЯ ХОХЛИНА, СУКА. Какие-же хохлы дененераты, пиздец просто.
Вероятность токсичности 0.92

Текст:  Надо признать, хохлы унизили пидорах итт.
Вероятность токсичности 0.86

Текст:  У него хохлы были в предках.
Вероятность токсичности 0.86

Текст:  Это же хохлы для битья на зарплате. Там все по сценарию.
Вероятность токсичности 0.86

Текст:  Блядь, хохлы, вы бы хоть с проксей писали, няши тупые.
Вероятность токсичности 0.86

Текст:  Нахуй иди, щенок. мимо нульчер 2010
Вероятность токсичности 0.85

Текст:  Пиздец у быдла с пикабу сначала горело от негров на нулевой, теперь от скримеров, куда я нахуй попал, ебаные животные это БЭ, ЭТО РАНДОМ СУЧАРА, ТАМ НЕ ДОЛЖНО БЫТЬ ПРАВИЛ, ПОШЕЛ НАХУЙ
Вероятность токсичности 0.84

Текст:  Плюсы: -Какие же хохлы дегенераты, пиздец просто. Минусы: -Какие же хохлв дегенераты, пиздец просто.
Вероятность токсичности 0.83

Текст:  Не надо всю, Галицию нахуй надо слать, пусть пшеки забирают даром.


Самое смешное, что почти все про хохлов) Тексты токсичные, хотя есть один без ругательств, но со словом "хохлы", но модель поняла, что там где есть "хохлы", то это коммент токсика, хотя по факту это может быть ошибочно и будет работать не везде. 

ПС в конце будет дополнение по этому поводу.

In [870]:
x_train, x_test, y_train, y_test = train_test_split(data['comment'], data['toxic'], test_size=0.1, shuffle=True, stratify=data['toxic'], random_state=42)

count_vectorizer = CountVectorizer(stop_words=russian_stopwords, 
                                   ngram_range=(1,1),
                                   max_df=0.999,
                                   min_df=0.001,
                                   tokenizer=my_tokenizer
                            )

x_train_vect = count_vectorizer.fit_transform(x_train)
x_test_vect = count_vectorizer.transform(x_test)

In [863]:
rf = RandomForestClassifier(n_estimators=500,
                            max_features='log2',
                            min_samples_split=3
                           )
rf.fit(x_train_vect, y_train)

y_preds = rf.predict(x_test_vect)

In [864]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85       959
         1.0       0.70      0.73      0.72       483

    accuracy                           0.81      1442
   macro avg       0.78      0.79      0.78      1442
weighted avg       0.81      0.81      0.81      1442



In [865]:
x_test = x_test.reset_index()
x_test['pred_proba'] = rf.predict_proba(x_test_vect)[:, 1]
x_test.drop(['index'], axis=1, inplace=True)

In [866]:
toxic_texts_rf = x_test[['comment', 'pred_proba']].sort_values(by='pred_proba', ascending=False).head(10).values

for text, prob in toxic_texts_rf:
    print('Текст: ', text)
    print('Вероятность токсичности', round(prob, 2))
    print()

Текст:  То на них пьяное быдло просто так напало и они его убили пьяное быдло плиз
Вероятность токсичности 1.0

Текст:  Надо признать, хохлы унизили пидорах итт.
Вероятность токсичности 1.0

Текст:  БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. НЕПРОБИВАЕМАЯ ХОХЛИНА, СУКА. Какие-же хохлы дененераты, пиздец просто.
Вероятность токсичности 1.0

Текст:  Это же хохлы для битья на зарплате. Там все по сценарию.
Вероятность токсичности 1.0

Текст:  Ты дебил? Как тогда те самолеты, которые разбились, поднялись в воздух?
Вероятность токсичности 1.0

Текст:  У него хохлы были в предках.
Вероятность токсичности 1.0

Текст:  Хохлоублюдки ответят за донбасс!!!
Вероятность токсичности 1.0

Текст:  Хул сука пугаешь падла
Вероятность токсичности 1.0

Текст:  Приятного сука аппетита.
Вероятность токсичности 1.0

Текст:  Все, едем в пендостан ебать тупых пендосских шлюх.
Вероятность токсичности 1.0



Пересекающиеся комменты:

In [867]:
set(toxic_texts_rf[:, 0]) & set(toxic_texts_lr[:, 0])

{'БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. НЕПРОБИВАЕМАЯ ХОХЛИНА, СУКА. Какие-же хохлы дененераты, пиздец просто.',
 'Надо признать, хохлы унизили пидорах итт.',
 'У него хохлы были в предках.',
 'Это же хохлы для битья на зарплате. Там все по сценарию.'}

Различные комменты:

In [868]:
set(toxic_texts_rf[:, 0]) - set(toxic_texts_lr[:, 0])

{'Все, едем в пендостан ебать тупых пендосских шлюх.',
 'Приятного сука аппетита.',
 'То на них пьяное быдло просто так напало и они его убили пьяное быдло плиз',
 'Ты дебил? Как тогда те самолеты, которые разбились, поднялись в воздух?',
 'Хохлоублюдки ответят за донбасс!!!',
 'Хул сука пугаешь падла'}

In [869]:
set(toxic_texts_lr[:, 0]) - set(toxic_texts_rf[:, 0])

{'Блядь, хохлы, вы бы хоть с проксей писали, няши тупые.',
 'Нахуй иди, щенок. мимо нульчер 2010',
 'Не надо всю, Галицию нахуй надо слать, пусть пшеки забирают даром.',
 'Пиздец у быдла с пикабу сначала горело от негров на нулевой, теперь от скримеров, куда я нахуй попал, ебаные животные это БЭ, ЭТО РАНДОМ СУЧАРА, ТАМ НЕ ДОЛЖНО БЫТЬ ПРАВИЛ, ПОШЕЛ НАХУЙ',
 'Плюсы: -Какие же хохлы дегенераты, пиздец просто. Минусы: -Какие же хохлв дегенераты, пиздец просто.',
 'Что? Какой денежный петух? Нет у хохлов своих сказок, коммуняки потом навыдумали хохлам культурку. А по факту то нация свинопасов и быдла, не городская. В том же Львове жили в основном поляки и евреи - городское население, а хохлы так. Быдло и селюки.'}

В принципе везде токсичные.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [871]:
feat_imp_rf = pd.DataFrame(np.c_[np.array(list(count_vectorizer.get_feature_names())), rf.feature_importances_], columns=['token', 'coef'])

In [872]:
feat_imp_rf['coef'] = feat_imp_rf['coef'].astype('float32')

In [873]:
feat_imp_rf.sort_values(by='coef', ascending=False).head(5)

,token,coef
2116,хохлы,0.010296
2115,хохлов,0.009894
1007,нахуй,0.007582
141,блядь,0.005671
1264,пиздец,0.005350


In [874]:
feat_imp_lr = pd.DataFrame(np.c_[np.array(list(tfidf_vectorizer.get_feature_names())), lr.coef_[0]], columns=['token', 'coef'])

In [875]:
feat_imp_lr['coef'] = feat_imp_lr['coef'].astype('float32')

In [876]:
feat_imp_lr.sort_values(by='coef', ascending=False).head(5)

,token,coef
1911,хохлы,1.713794
1910,хохлов,1.635465
928,нахуй,1.453508
135,блядь,1.251428
1163,пиздец,1.189118


Слова те же самые у обоих классификаторов, по поводу хохлов гипотеза подтвердилась, но в любом случае это слово возможно использовать вне контекста токсичного комментария (хотя тут не факт, видимо уже это понятие настолько трансформировалось, что упоминается только в негативном ключе).